# In silicoChIP on peak set from in vitro blood data

In [1]:
###################
## Load packages
###################
suppressPackageStartupMessages({
    library(scran)
    library(scater)
    library(Seurat)
    library(ArchR)
})
source('/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/code/revision/insilico_chip/utils.R')


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
###################
## I/O
###################
io = list()
io$basedir = '/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/'

## Single-cell
io$RNA_sce = file.path(io$basedir,"data/processed/rna/SingleCellExperiment.rds")
io$meta = file.path(io$basedir,"results/atac/archR/qc/sample_metadata_after_qc.txt.gz")

# ArchR vitro
io$archrvitro = file.path("/rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/processed/atac/archR")
# ArchR vivo
io$archrvivo = file.path(io$basedir,"data/processed/atac/archR")

# output
io$outdir = file.path("/rds/project/rds-SDzz0CATGms/users/bt392/09_Eomes_invitro_blood/results/multiome_atlas")
dir.create(io$outdir, recursive=TRUE, showWarnings =FALSE)

In [3]:
meta = fread(io$meta)[pass_rnaQC == T & pass_atacQC == T & doublet_call == F & !sample %in% c('E8.5_CRISPR_T_KO', 'E8.5_CRISPR_T_WT')]
nrow(meta)

[1] 45713

In [4]:
archrvivo = loadArchRProject(io$archrvivo)[meta$cell]
archrvitro = loadArchRProject(io$archrvitro)

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [5]:
archrvivo


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /rds/project/rds-SDzz0CATGms/users/bt392/atlasses/gastrulation_multiome/data/processed/atac/archR 
samples(11): E7.5_rep1 E7.5_rep2 ... E8.5_CRISPR_T_KO E8.5_CRISPR_T_WT
sampleColData names(1): ArrowFiles
cellColData names(35): Sample TSSEnrichment ... ReadsInPeaks FRIP
numberOfCells(1): 45713
medianTSS(1): 16.355
medianFrags(1): 30489

In [6]:
# Add in vitro peak matrix to vivo ArchR object
vitroPeaks = getPeakSet(archrvitro)

In [7]:
addArchRThreads(10)
archrvivo = addFeatureMatrix(
              input = archrvivo,
              features = vitroPeaks,
              matrixName = "vitroPeakMatrix",
              ceiling = 4,
              binarize = FALSE,
              verbose = TRUE,
              threads = getArchRThreads(),
              parallelParam = NULL,
              force = TRUE,
              logFile = createLogFile("vitroPeakMatrix")
            )

Setting default number of Parallel threads to 10.

ArchR logging to : ArchRLogs/ArchR-vitroPeakMatrix-20a16c52c1c860-Date-2024-04-26_Time-17-11-09.log
If there is an issue, please report to github with logFile!

2024-04-26 17:11:11 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-vitroPeakMatrix-20a16c52c1c860-Date-2024-04-26_Time-17-11-09.log



In [8]:
addArchRThreads(5)
atac.sce = getMatrixFromProject(archrvivo, useMatrix = 'vitroPeakMatrix')

Setting default number of Parallel threads to 5.

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-20a16c7d29a448-Date-2024-04-26_Time-17-14-51.log
If there is an issue, please report to github with logFile!

2024-04-26 17:15:56 : Organizing colData, 1.089 mins elapsed.

2024-04-26 17:15:57 : Organizing rowData, 1.095 mins elapsed.

2024-04-26 17:15:57 : Organizing rowRanges, 1.095 mins elapsed.

2024-04-26 17:15:57 : Organizing Assays (1 of 1), 1.095 mins elapsed.

2024-04-26 17:16:28 : Constructing SummarizedExperiment, 1.621 mins elapsed.

2024-04-26 17:17:06 : Finished Matrix Creation, 2.25 mins elapsed.



In [11]:
## High resolution clustering to determine correlations between ATAC and RNA
## Note: I'm not performing any batch correction, Clusters will likely have some form of sample-specificity, but it is unlikely that this will affect downstream results as they will still capture cell-type within those samples as well
# Load SingleCellExperiment
rna.sce <- readRDS(io$RNA_sce)
#rna.sce = rna.sce[,match(meta$cell, colnames(rna.sce))]
# Make sure that samples are consistent
cells <- intersect(colnames(rna.sce),colnames(atac.sce))
rna.sce <- rna.sce[,cells]
atac.sce <- atac.sce[,cells]

In [12]:
rna.sce = logNormCounts(rna.sce)

In [13]:
unique(rna.sce$sample)

[1] "E7.5_rep1"  "E7.5_rep2"  "E7.75_rep1" "E8.0_rep1"  "E8.0_rep2" 
[6] "E8.5_rep1"  "E8.5_rep2"  "E8.75_rep1" "E8.75_rep2"

In [14]:
# RNA dimreduction
decomp <- modelGeneVar(rna.sce, block = rna.sce$sample)
decomp <- decomp[decomp$mean > 0.01,]
hvgs <- decomp[order(decomp$FDR),] %>% head(n=5000) %>% rownames

sce_filt <- rna.sce[hvgs,]
sce_filt <- runPCA(sce_filt, ncomponents = 40, ntop=5000)

In [15]:
# Create Seurat object with PCA 
pca <- reducedDim(sce_filt,"PCA")
counts = matrix(ncol=length(colnames(rna.sce)), nrow=1)
colnames(counts) = colnames(rna.sce)
rownames(counts) = 'x'
seurat = CreateSeuratObject(counts)                
seurat[["pca"]] <- CreateDimReducObject(embeddings = pca, key = "PC_", assay = DefaultAssay(seurat))

In [16]:
# high clustering resolution instead of metacells
library(future)
plan("multisession", workers = 6)
seurat <- FindNeighbors(seurat, dims = 1:40)
seurat <- FindClusters(seurat, resolution = 20)

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 45707
Number of edges: 1531796

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7127
Number of communities: 222
Elapsed time: 4 seconds


In [17]:
clusters = as.data.table(seurat@meta.data, keep.rownames=T) %>% .[,c('rn', 'seurat_clusters')] %>% setnames(c('cell', 'cluster'))

In [18]:
length(unique(clusters$cluster))

[1] 222

In [19]:
summary(colnames(rna.sce) == clusters$cell)

   Mode    TRUE 
logical   45707 

In [20]:
library(BiocParallel)
BPPARAM = MulticoreParam(detectCores()-2)

In [21]:
summary(colnames(rna.sce) == colnames(atac.sce))

   Mode    TRUE 
logical   45707 

In [22]:
summary(clusters$cell == colnames(rna.sce))

   Mode    TRUE 
logical   45707 

In [23]:
# Pseudobulk by clusters
rna.pb = aggregateAcrossCells(rna.sce, id=clusters$cluster, BPPARAM = BPPARAM)
atac.pb = aggregateAcrossCells(atac.sce, use.assay.type = 'vitroPeakMatrix', id=clusters$cluster, BPPARAM = BPPARAM)

In [24]:
assay(rna.pb,"logcounts") <- log(1e6*(sweep(assay(rna.pb),2,colSums(assay(rna.pb),na.rm=T),"/"))+1) 
assay(atac.pb,"logcounts") <- log(1e6*(sweep(assay(atac.pb),2,colSums(assay(atac.pb),na.rm=T),"/"))+1)

# Cisbp

In [25]:
## Get motif matches for example motifs in peaks 
## Adapted from: https://rdrr.io/github/GreenleafLab/ArchR/src/R/AnnotationPeaks.R
# Load motifs
library(motifmatchr)
library(chromVARmotifs)

data("mouse_pwms_v2")
motifs <- mouse_pwms_v2
obj <- ArchR:::.summarizeChromVARMotifs(motifs)
motifs <- obj$motifs
motifSummary <- obj$motifSummary
names(motifs) = gsub( 'Tcfap', 'tfap', names(motifs))

# Find scores
motifmatcher.se <-  motifmatchr::matchMotifs(pwms = motifs, 
                                        subject = vitroPeaks, 
                                        genome = "mm10", 
                                        out = 'scores',
                                        p.cutoff = 5e-05,
                                        w = 7) 




Attaching package: ‘XVector’


The following object is masked from ‘package:purrr’:

    compact


The following object is masked from ‘package:plyr’:

    compact



Attaching package: ‘Biostrings’


The following object is masked from ‘package:grid’:

    pattern


The following object is masked from ‘package:base’:

    strsplit




In [26]:
# Add peak names to objects
peaks = paste0(seqnames(vitroPeaks), ':', ranges(vitroPeaks))
rownames(atac.pb) = peaks
rownames(motifmatcher.se) = peaks

motif2gene.dt = data.table(motif = colnames(motifmatcher.se),
                           gene = toupper(str_split(colnames(motifmatcher.se), '_') %>% map_chr(1)))

In [27]:
###################
## Correlate TF-expr & Region-accessibility
###################  
a = Sys.time()
tf2peak_cor.se = cor_TF_acc(rna.sce = rna.pb, 
                          atac.sce = atac.pb, 
                          TFs_filt = NULL, #colData(motifmatcher.se)$name,  # c('SOX17', 'MYBL1') -> test runs
                          motifmatcher.se = motifmatcher.se, 
                          motif2gene.dt = motif2gene.dt,
                          correlation_method = "pearson", 
                          remove_motifs = NULL,
                          cores=1) # got an error this time with more cores
b = Sys.time()
b-a
# Make sure no multicore errors!!!

[1] "Removing 5 TFs that have duplicated gene-motif pairs:\nFOXL1, GM4881, GM5294, NFIL3, ZFP263"


Correlate TF-expr & Region-accessibility for 738 TFs



SOX17 
MYBL1 
MSC 
TFAP2D 
TFAP2B 
ARID5A 
NPAS2 
RFX8 
POU3F3 
STAT4 
STAT1 
HSFY2 


Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”


KLF7 
CREB1 
FEV 
PAX3 
SP110 
SP140 
SP100 
GBX2 
TWIST2 
GLI2 
EN1 
ELK4 
SOX13 
MYOG 
ELF3 
ZFP281 
NR5A2 
LHX9 
LHX4 
PRRX1 
TBX19 
POU2F1 
RXRG 
LMX1A 
PBX1 
ATF6 
NR1I3 
USF1 
NHLH1 
AHCTF1 
MIXL1 
HLX 
ESRRG 
BATF3 
ATF3 
IRF6 
GATA3 
PTF1A 
TBPL2 
PAX8 
LHX3 
RXRA 
GFI1B 
BARHL1 
PRRX2 
NAIF1 
LMX1B 
PBX3 
LHX6 
ZBTB6 
LHX2 
NR5A1 
NR6A1 
NR4A2 
TBR1 
SP5 
DLX1 
DLX2 
SP3 
SP9 
ATF2 
EVX2 
HOXD13 
HOXD12 


...

TCF7 
IRF1 
HAND1 
SREBF1 
HES7 
TRP53 
SOX15 
ZBTB4 
YBX2 
BCL6B 
ZFP3 
MNT 
HIC1 
SEBOX 
HNF1B 
LHX1 
TBX2 
TBX4 
HLF 
DLX3 
DLX4 
ZFP652 
HOXB13 
HOXB9 
HOXB8 
HOXB7 
HOXB6 
HOXB5 
HOXB4 
HOXB3 
HOXB2 
HOXB1 
NFE2L1 
SP2 
SP6 
TBX21 
NEUROD2 
THRA 
NR1D1 
RARA 
STAT5B 
STAT5A 
STAT3 
MLX 
ETV4 
MEOX1 
SOX9 
FOXJ1 
MAFG 
FOXK2 
OSR1 
MYCN 
E2F6 
GRHL1 
KLF11 
ID2 
SOX11 
MYT1L 
HBP1 
FERD3L 
TWIST1 
AHR 
MEOX2 
ETV1 
FOXG1 
NPAS3 
PAX9 
FOXA1 
SIX6 
SIX1 
SIX4 
ESR2 
ZBTB1 
MAX 
ZFP410 
VSX2 
FOS 
JDP2 
BATF 
ESRRB 
FOXN3 
GSC 
BCL11B 
YY1 
SP4 
SP8 
KLF6 
GLI3 
POU6F2 
ZKSCAN3 
ZKSCAN4 
SOX4 
E2F3 
IRF4 
FOXQ1 
FOXF2 
FOXC1 
RREB1 
TFAP2A 
GCM2 
HIVEP1 
ID4 
BARX1 
MSX2 
PITX1 
NEUROG1 
SMAD5 
ZFP369 
IRX1 
IRX2 
IRX4 
AHRR 
NR2F1 
MEF2C 
SPZ1 
OTP 
FOXD1 
ISL1 
FEZF2 
RARB 
THRB 
NR1D2 
HESX1 
PRRXL1 
OTX2 
ZFP219 
CEBPE 
HOMEZ 
NRL 
IRF9 
NFATC4 
GATA4 
SOX7 
HMBOX1 
EGR3 
ZFP957 
ELF1 
KLF5 
KLF12 
POU4F1 
SOX21 
ZIC5 
ZIC2 
DNAJC21 
OSR2 
KLF10 
MYC 
MAFA 
SCX 
HSF1 
SCRT1 
FOXH

Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”


ELF4 
ZIC3 
SOX3 
MECP2 
ARX 
ZFX 
AR 
FOXO4 
CDX4 
TBX22 
POU3F4 
HDX 
ZFP711 
ESX1 
KLF8 
MBTPS2 
ZFY1 
SRY 


Warning message in cor(x, y, use = use, method = method):
“the standard deviation is zero”
Warning message in `[.data.table`(., , `:=`(peak, rownames(atac.sce))):
“Invalid .internal.selfref detected and fixed by taking a (shallow) copy of the data.table so that := can add this new column by reference. At an earlier point, this data.table has been copied by R (or was created manually using structure() or similar). Avoid names<- and attr<- which in R currently (and oddly) may copy the whole data.table. Use set* syntax instead to avoid copying: ?set, ?setnames and ?setattr. If this message doesn't help, please report your use case to the data.table issue tracker so the root cause can be fixed or this message improved.”


Time difference of 6.111171 mins

In [30]:
atac.sce

class: RangedSummarizedExperiment 
dim: 234908 45707 
metadata(0):
assays(1): vitroPeakMatrix
rownames: NULL
rowData names(1): idx
colnames(45707): E7.5_rep1#AAACAGCCATCCTGAA-1
  E7.5_rep1#AAACAGCCATGCTATG-1 ... E8.75_rep2#TTTGTTGGTTCACTGT-1
  E8.75_rep2#TTTGTTGGTTGAGCCG-1
colData names(35): BlacklistRatio nDiFrags ... ReadsInPeaks FRIP

In [31]:
atac.pb

class: RangedSummarizedExperiment 
dim: 234908 222 
metadata(0):
assays(2): vitroPeakMatrix logcounts
rownames(234908): chr1:3003470-3004070 chr1:3035598-3036198 ...
  chrX:169925416-169926016 chrX:169937236-169937836
rowData names(1): idx
colnames(222): 0 1 ... 220 221
colData names(37): BlacklistRatio nDiFrags ... ids ncells

In [72]:
######################################
## Create virtual chip-seq library 
######################################
a = Sys.time()
motifmatcher_chip.se = silico_chip(atac.sce = atac.pb, 
                           tf2peak_cor.se = tf2peak_cor.se,
                           motifmatcher.se = motifmatcher.se,
                           motif2gene.dt = motif2gene.dt, 
                           min_number_peaks = 50,
                           TFs_filt = NULL, 
                           remove_motifs = NULL,
                           cores = 6)
b = Sys.time()
b-a

[1] "Number of peaks: 234908"
[1] "Predicting TF binding sites..."
[1] "Number of TFs: 738"
[1] "Updating motifmatchr results using the virtual ChIP-seq library..."


Time difference of 45.73569 secs

In [81]:
saveRDS(motifmatcher_chip.se$motifmatcher_chip.se, file.path(io$outdir, 'vitro_peaks_silicoChIP_cisbp.rds'))

# Jaspar2020

In [55]:
JASPAR2020::JASPAR2020

An object of class "JASPAR2020"
Slot "db":
[1] "/rds/project/rds-SDzz0CATGms/users/bt392/miniconda3/envs/env_all/lib/R/library/JASPAR2020/extdata/JASPAR2020.sqlite"


In [47]:
## Get motif matches for example motifs in peaks 
## Adapted from: https://rdrr.io/github/GreenleafLab/ArchR/src/R/AnnotationPeaks.R
# Load motifs
library(motifmatchr)
library(JASPAR2020)

args <- list(species = "Homo sapiens", collection = "CORE") # Needs to be human?
motifs <- TFBSTools::getMatrixSet(JASPAR2020::JASPAR2020, args)
obj <- ArchR:::.summarizeJASPARMotifs(motifs)
motifs <- obj$motifs
motifSummary <- obj$motifSummary
#names(motifs) = gsub( 'TBXT', 'T', names(motifs)) # Rename T to fit

# Find scores
motifmatcher.se <-  motifmatchr::matchMotifs(pwms = motifs, 
                                        subject = vitroPeaks, 
                                        genome = "mm10", 
                                        out = 'scores',
                                        p.cutoff = 5e-05,
                                        w = 7) 

In [52]:
# Add peak names to objects
peaks = paste0(seqnames(vitroPeaks), ':', ranges(vitroPeaks))
rownames(atac.pb) = peaks
rownames(motifmatcher.se) = peaks

motif2gene.dt = data.table(motif = colnames(motifmatcher.se),
                           gene = toupper(str_split(colnames(motifmatcher.se), '_') %>% map_chr(1)))

In [53]:
###################
## Correlate TF-expr & Region-accessibility
###################  
a = Sys.time()
tf2peak_cor.se = cor_TF_acc(rna.sce = rna.pb, 
                          atac.sce = atac.pb, 
                          TFs_filt = NULL, #colData(motifmatcher.se)$name,  # c('SOX17', 'MYBL1') -> test runs
                          motifmatcher.se = motifmatcher.se, 
                          motif2gene.dt = motif2gene.dt,
                          correlation_method = "pearson", 
                          remove_motifs = NULL,
                          cores=6) # got an error this time with more cores
b = Sys.time()
b-a

[1] "Removing 0 TFs that have duplicated gene-motif pairs:\n"


Correlate TF-expr & Region-accessibility for 504 TFs



Time difference of 55.27338 secs

In [54]:
######################################
## Create virtual chip-seq library 
######################################
a = Sys.time()
motifmatcher_chip.se = silico_chip(atac.sce = atac.pb, 
                           tf2peak_cor.se = tf2peak_cor.se,
                           motifmatcher.se = motifmatcher.se,
                           motif2gene.dt = motif2gene.dt, 
                           min_number_peaks = 50,
                           TFs_filt = NULL, 
                           remove_motifs = NULL,
                           cores = 6)
b = Sys.time()
b-a

[1] "Number of peaks: 234908"
[1] "Predicting TF binding sites..."
[1] "Number of TFs: 504"
[1] "Updating motifmatchr results using the virtual ChIP-seq library..."


Time difference of 29.37032 secs

In [56]:
saveRDS(motifmatcher_chip.se$motifmatcher_chip.se, file.path(io$outdir, 'vitro_peaks_silicoChIP_jaspar2020.rds'))